In [1]:
import os
import sys
import pathlib
import platform
import urllib.request
import shutil

from typing import (
    Dict,
    List
)

In [2]:
__file__ = 'db_tests.ipynb'

In [3]:
mod_path: str = os.path.join(str(pathlib.Path(os.path.abspath(__file__)).parents[2]))
sys.path.append(mod_path)

In [4]:
from convert_source.cs_utils.fileio import Command

In [5]:
from convert_source.cs_utils.database import (
    construct_db_dict,
    create_db,
    insert_row_db,
    get_len_rows,
    get_file_id,
    update_table_row,
    export_dataframe,
    export_scans_dataframe,
    _get_dir_relative_path,
    _export_tmp_bids_df,
    export_bids_scans_dataframe,
    query_db,
    _zeropad
)

In [44]:
# Test variables
scripts_dir: str = os.path.abspath(os.path.join(os.path.dirname(__file__),'helper.scripts'))
test_config1: str = os.path.join(scripts_dir,'test.1.config.yml')
test_config2: str = os.path.join(scripts_dir,'test.2.config.yml')

data_dir: str = os.path.abspath(os.path.join(os.path.dirname(__file__),'test.study_dir'))
dcm_test_data: str = os.path.join(data_dir,'TEST001-UNIT001','data.dicom','ST000000')

test_infile: str = os.path.join(scripts_dir,'test.orig_subject_id.txt')
test_mapfile: str = os.path.join(scripts_dir,'test.map_subject_id.txt')

out_dir: str = os.path.join(os.getcwd(),'test.database.study)
misc_dir: str = os.path.join(out_dir,'.misc')
test_db: str = os.path.join(misc_dir,'test.study.db')

SyntaxError: EOL while scanning string literal (<ipython-input-44-0c96371ac05f>, line 12)

In [7]:
def test_download_prog():
    class PlatformInferError(Exception):
        pass

    if platform.system().lower() == 'darwin':
        url: str = "https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20201102/dcm2niix_mac.zip"
        file_name: str = "dcm2niix_mac.zip"
    elif platform.system().lower() == 'windows':
        url: str = "https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20201102/dcm2niix_win.zip"
        file_name: str = "dcm2niix_win.zip"
    elif platform.system().lower() == 'linux':
        url: str = "https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20201102/dcm2niix_lnx.zip"
        file_name: str = "dcm2niix_lin.zip"
    else:
        raise PlatformInferError("Unable to infer this platform's operating system.")

    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    
    assert os.path.exists(file_name) == True
    
    if platform.system().lower() == 'windows':
        extract: Command = Command("tar")
        extract.cmd_list.append("-zxvf")
        extract.cmd_list.append(file_name)
        extract.run()
    else:
        extract: Command = Command("unzip")
        extract.cmd_list.append(file_name)
        extract.run()

    os.remove(file_name)
    assert os.path.exists(file_name) == False

In [8]:
def test_extract_data():
    dcm_data: str = os.path.join(data_dir,'TEST001-UNIT001','data.dicom','data.1.tar.gz')
    extract: Command = Command("tar")
    extract.cmd_list.append("-zxvf")
    extract.cmd_list.append(dcm_data)
    extract.cmd_list.append("-C")
    extract.cmd_list.append(
        os.path.dirname(dcm_data)
    )
    extract.run()
    assert os.path.exists(dcm_test_data) == True

In [9]:
# Only need to run once
test_download_prog()
test_extract_data()

ERROR: x ./ST000000/
x ./ST000000/.misc/
x ./ST000000/SE000000/
x ./ST000000/SE000001/
x ./ST000000/SE000002/
x ./ST000000/SE000002/MR000000.dcm
x ./ST000000/SE000002/MR000002.dcm
x ./ST000000/SE000002/MR000004.dcm
x ./ST000000/SE000002/MR000005.dcm
x ./ST000000/SE000002/MR000006.dcm
x ./ST000000/SE000002/MR000010.dcm
x ./ST000000/SE000002/MR000011.dcm
x ./ST000000/SE000002/MR000013.dcm
x ./ST000000/SE000002/MR000015.dcm
x ./ST000000/SE000001/MR000001.dcm
x ./ST000000/SE000001/MR000005.dcm
x ./ST000000/SE000001/MR000009.dcm
x ./ST000000/SE000001/MR000012.dcm
x ./ST000000/SE000001/MR000016.dcm
x ./ST000000/SE000001/MR000017.dcm
x ./ST000000/SE000001/MR000020.dcm
x ./ST000000/SE000001/MR000024.dcm
x ./ST000000/SE000000/MR000002.dcm
x ./ST000000/.misc/ctkDICOM.sql
x ./ST000000/.misc/ctkDICOMTagCache.sql
x ./ST000000/.misc/dicom/
x ./ST000000/.misc/thumbs/
x ./ST000000/.misc/thumbs/1.2.840.113619.2.243.6074146108103184.41976.4343.2084088/
x ./ST000000/.misc/thumbs/1.2.840.113619.2.243.6074

In [40]:
def test_create_db():
    if os.path.exists(misc_dir):
        pass
    else:
        os.makedirs(misc_dir)
    
    assert os.path.exists(create_db(database=test_db)) == True
    shutil.rmtree(out_dir)
    assert os.path.exists(out_dir) == False

In [43]:
test_create_db()

In [37]:
test_file_1: str = os.path.join(
    data_dir,
    'TEST001-UNIT001',
    'data.dicom',
    'ST000000',
    'SE000001',
    'MR000009.dcm'
)

test_file_2: str = os.path.join(
    data_dir,
    'TEST001-UNIT001',
    'data.parrec',
    'AXIAL.PAR'

)

test_file_3: str = os.path.join(
    data_dir,
    'TEST001-UNIT001',
    'data.nifti',
    'FLAIR.nii.gz'
)

In [ ]:
# TODO: 
#   Create file path/name truncation function for DICOMs
#       for database query consistency.

In [45]:
if os.path.exists(misc_dir):
    pass
else:
    os.makedirs(misc_dir)

create_db(database=test_db)

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study.link/.misc/test.study.db'

In [ ]:
construct_db_dict()